# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter von Domherren einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: 
`domherr.tbl_domherren` und `domherr.tbl_domherren_aemter`

[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Archidiakonat](#Archidiakonat)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt) (hinfällig?)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [20]:
item_type_id = 5

5

In [10]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-10-19"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-10-19"

In [11]:
out_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

Die Ämter in Tabelle `role` werden im Notebook "Strukturdaten" eingelesen.

## Amtsperiode

In [13]:
using CSV, DataFrames

In [12]:
# Quelle CSV
filename = joinpath(data_path, "tbl_Domherren_Aemter.csv")
df_pr = CSV.read(filename, DataFrame);

In [11]:
# Quelle Datenbank
sql = "SELECT * FROM domherr.tbl_domherren_aemter"
df_pr = Wds.sql_df(sql);

In [13]:
Wds.clean_up!(df_pr);

Ignoriere Einträge wo sowohl Amtsart als auch ID_Amt_norm fehlen.

In [14]:
size(df_pr)

(38919, 20)

In [15]:
one_valid(a, b) = !ismissing(a) | !ismissing(b)

one_valid (generic function with 1 method)

In [16]:
subset!(df_pr, [:Amtsart, :ID_Amt_norm] => ByRow(one_valid));

In [17]:
df_pr[101:105, [:ID_Amt, :Amtsart, :ID_Amt_norm, :Amtsbeginn, :Amtsende]]

,ID_Amt,Amtsart,ID_Amt_norm,Amtsbeginn,Amtsende
,Int64,String?,Int64?,String?,String?
1,85693,missing,22,1551,missing
2,85694,missing,19,1552,1581
3,85695,missing,22,1596,missing
4,85696,missing,19,1605,1629
5,85697,missing,19,1641,1679


### Domherren zuordnen

In [18]:
transform!(df_pr, :ID_Domherr => ByRow(string) => :id_in_source);

In [21]:
table_name = "item";
sql = "SELECT id AS person_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)"
df_idx = Wds.sql_df(sql);

Lösche Einträge ohne Wert für `ID_Domherr`

In [22]:
dropmissing!(df_pr, :ID_Domherr);

In [23]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

In [24]:
df_idx_mg = filter(:person_id => ismissing, df_prp);

In [25]:
size(df_idx_mg)

(0, 22)

In [87]:
df_idx_mg[:, [:ID_Domherr, :ID_Amt, :Amtsart]]

,ID_Domherr,ID_Amt,Amtsart
,Int64,Int64,String?


### Rollen/Ämter zuordnen

In [26]:
sql = "SELECT name as role_name, role.id as role_id, item.id_in_source as role_id_in_source
FROM role JOIN item ON item.id = role.id"
df_role = Wds.sql_df(sql);

Werden alle Ämter gefunden?

In [27]:
role_dh = unique(string.(df_prp.ID_Amt_norm));

In [28]:
match_mg = setdiff(role_dh, df_role.role_id_in_source)

1-element Vector{String}:
 "missing"

In [29]:
to_string_maybe(a) = ismissing(a) ? a : string(a)

to_string_maybe (generic function with 1 method)

In [30]:
transform!(df_prp, :ID_Amt_norm => ByRow(to_string_maybe) => :match_role_id);

In [31]:
df_prp_r = leftjoin(df_prp, df_role, on = :match_role_id => :role_id_in_source, matchmissing = :notequal);

In [32]:
count(ismissing, df_prp_r.role_id), count(ismissing, df_prp_r.Amtsart)

(1161, 6886)

In [33]:
df_prp_r_mg_rid = filter(:role_id => ismissing, df_prp_r);

In [37]:
unique(df_prp_r_mg_rid.Amtsart)

87-element Vector{Union{Missing, String}}:
 "Dombaumeister"
 "Wirtschaftsverwalter"
 "Inhaber der officium minor"
 "officiales"
 "Mitglied"
 "Ecclesiasticus"
 "Domprior/seit seit 1504 Dekane"
 "Domprior/seit 1504 Dekan"
 "Domkustoden/Domthesaurare"
 "Präsenzmeister"
 "Präsenzkeller"
 "Koadjutor und Administrator"
 "Providierter Domherr/Domscholast"
 ⋮
 "vorwarer"
 "Vorsteher, Amtmann, confessarius, Gemeindepfarrer"
 "vormund"
 "bischöflicher Vogt"
 "Amtsverwalter"
 "Domina bzw. Äbtissin"
 "Drost u. Amtsschreiber"
 "Amtspächter bzw. -verwalter"
 "Lehnpropst"
 "Vorsteher (Ordenskaplan)"
 "Weltgeistlicher"
 "Rectores scholarum"

*2022-10-19* Was machen wir mit diesen Einträgen?

*2022-03-16* TODO übernimm die Rollenbezeichnung `person_role.role_name` aus der Tabelle `role`.

### Bistümer zuordnen

In [33]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

In [34]:
df_prp_r = leftjoin(df_prp_r, df_dioc, on = :Bistum => :diocese_name, matchmissing = :notequal);

In [35]:
idx_diocese = .!ismissing.(df_prp_r[!, :diocese_id]);

In [36]:
sum(idx_diocese)

1042

### Instutionen zuordnen
Klöster, Domstifte

In [37]:
table_name = "institution";
sql = "SELECT id AS institution_id, name, id_gsn FROM $(table_name)"
df_inst = Wds.sql_df(sql);

In [38]:
df_prp_r = leftjoin(df_prp_r, df_inst, on = :ID_Kloster => :id_gsn, matchmissing = :notequal);

Version mit eigener lookup-Funktion

In [39]:
lookup_inst = Dict(df_inst.id_gsn .=> df_inst.institution_id);

In [40]:
length(lookup_inst)

5286

In [41]:
get_inst_id(id_gsn) = get(lookup_inst, id_gsn, missing)

get_inst_id (generic function with 1 method)

In [42]:
get_inst_id(318)

79619

In [43]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_id) => :institution_id);

In [44]:
lookup_inst_name = Dict(df_inst.id_gsn .=> df_inst.name);

In [45]:
get_inst_name(id_gsn) = get(lookup_inst_name, id_gsn, missing)

get_inst_name (generic function with 1 method)

In [46]:
get_inst_name(318)

"Benediktinerinnenkloster Neuenwalde"

In [47]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_name) => :institution_name);

In [48]:
df_mg_inst_id = subset(df_prp_r, :institution_id => ByRow(ismissing));

In [49]:
size(df_mg_inst_id, 1), size(df_prp_r, 1)

(1136, 38067)

### Numerische Daten eintragen

In [50]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
df_prp_r[!, :num_date_begin] .= num_date_begin.(df_prp_r[!, :Amtsbeginn]);

┌ Warning: Could only find year in 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1125
┌ Warning: Could only find year in 
│   s = t 1567
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = Bewerbung 1451
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = erneut 1239
└ @ WiagDataSetup C:\Users\georg\Documents\projekte

In [51]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper)
df_prp_r[!, :num_date_end] .= num_date_end.(df_prp_r[!, :Amtsende]);

┌ Warning: Could only find year in 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = t1529
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = t1547
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could not parse 
│   s = 66
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src

┌ Warning: Could only find year in 
│   s = resigniert 1417
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = resigniert 1442
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = resigniert 1519
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = resigniert 1489
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = resigniert 1524
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = resigniert 1568
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = priviert 1398
└ @ WiagData

In [52]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [53]:
size(df_info, 1)

31181

In [54]:
df_info[60:67, [:ID_Domherr, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

,ID_Domherr,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int64,String,String?,Int64?,String?,Int64?
1,14113,Domkustos (Domthesaurar),1778,1778,1792,1792
2,14115,Domkustos (Domthesaurar),1792,1792,1811,1811
3,14117,Domkantor,vor 1328,1278,missing,missing
4,14118,Domscholaster,vor 1328,1278,missing,missing
5,14122,Domkantor,1376,1376,1376,1376
6,14124,Domkantor,1405,1405,1419,1419
7,14128,Domkantor,1452,1452,1455,1455
8,13991,Domherr,1653,1653,1681,1681


In [55]:
size(df_prp)

(38067, 23)

*2022-05-20* Datenfehler in der Quelle korrigiert

In [126]:
function fix_date(s)
    if ismissing(s)
        return s
    end
    return replace(s, "1 Hälfte" => "1. Hälfte", "14 Jahr" => "14. Jahr", "Beginn16" => "Beginn 16")
end

fix_date (generic function with 1 method)

In [127]:
transform!(df_prp_r, :Amtsbeginn => ByRow(fix_date) => :Amtsbeginn);

In [128]:
transform!(df_prp_r, :Amtsende => ByRow(fix_date) => :Amtsende);

In [56]:
transform!(df_prp_r, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1553) 1560
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1574) 1593
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = t 1567
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (Bewerbung 1451)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539


┌ Warning: could not parse 
│   s = 1
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = ohne Angabe
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = [1284]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = [1309]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = ohne Angabe
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1553) 1560
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539


*2022-05-12* In `tbl_Domherren_Aemter` enthalten 200 Einträge ein `†`. Das wird vor dem Parsen entfernt.

In [57]:
transform!(df_prp_r, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: could not parse 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = t1529
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = t1547
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = [1223]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not 

┌ Warning: could not parse 
│   s = resigniert 1468
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = resigniert 1411
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = wahrscheinlich resigniert 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = permutiert 1423
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = resigniert 1417
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = resigniert 1442
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = resigniert 1519
└ @ WiagDataSetup C:\Users\georg\Documents\projekte

In [63]:
maximum(df_prp_r.date_sort_key)

9000900

In [58]:
replace_if_missing(a, b) = a == 9000900 ? b : a

replace_if_missing (generic function with 1 method)

In [59]:
transform!(df_prp_r, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

In [60]:
size(df_prp_r, 1), count(isequal(9000900), df_prp_r.date_sort_key)

(38067, 1767)

In [61]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [62]:
df_info[100:105, [:Amtsart, :ID_Amt_norm, :role_id, :Amtsbeginn, :num_date_begin, :num_date_end, :date_sort_key]]

,Amtsart,ID_Amt_norm,role_id,Amtsbeginn,num_date_begin,num_date_end,date_sort_key
,String,Int64?,Int32?,String?,Int64?,Int64?,Int64
1,Archidiakon,5,489576,vor 1707,1657,missing,1707100
2,Archidiakon,5,489576,vor 1755,1705,missing,1755100
3,Archidiakon,5,489576,1712,1712,1753,1712150
4,Archidiakon,5,489576,1784,1784,1791,1784150
5,Archidiakon,5,489576,1791,1791,1792,1791150
6,Archidiakon,5,489576,1757,1757,1771,1757150


Einträge für diesen Item_Typ löschen

In [63]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_id))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 38067, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer - auch für die identifizierten - den Namen des Bistums in die Tabelle.  
Schreibe für alle Institutionen - auch für die identifizieren - die Bezeichnung in die Tabelle.  
Schreibe für alle Ämter - auch für die identifizieren - die Bezeichnung in die Tabelle.

Erzeuge die IDs selbst, um sie für `person_role_property` zur Verfügung zu haben.

In [64]:
table_name = "person_role"
sql = "SELECT MAX(id) + 1 AS next_id FROM $(table_name)"
df_next_id = Wds.sql_df(sql)

,next_id
,Int64?
1,770296


In [65]:
next_id = df_next_id[1, :next_id] + 1

770297

In [66]:
df_prp_r[!, :id] .= collect(next_id:(next_id + size(df_prp_r, 1) - 1));

In [67]:
df_prp_r[1001:1007, [:id, :person_id, :ID_Domherr, :Amtsart, :ID_Amt]]

,id,person_id,ID_Domherr,Amtsart,ID_Amt
,Int64,Int32?,Int64,String?,Int64
1,771297,465525,44124,Domdekan,58662
2,771298,465524,44125,Domdekan,58663
3,771299,465523,44126,Domdekan,58664
4,771300,454650,25444,missing,58665
5,771301,455387,26250,missing,58666
6,771302,455184,26008,missing,58667
7,771303,455184,26008,missing,58668


In [68]:
columns = [
    :id => :id,
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :Institution => :institution_name,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [69]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prp_r, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: 30000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: Rows inserted: 38067
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


38067

Titularbistum ist noch nicht eingetragen

## Zeitspanne der Person

In [76]:
table_name = "person"
sql = "SELECT id, num_date_birth, num_date_death
FROM $(table_name) WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [77]:
size(df_date)

(26023, 3)

Zeitspanne der Ämter

In [78]:
df_pr_begin_0 = dropmissing(df_prp_r, :num_date_begin)
df_pr_begin = groupby(select(df_pr_begin_0, :person_id, :num_date_begin), :person_id)

,person_id,num_date_begin
,Int32?,Int64
1,443845,1680
2,443845,1678
3,443845,1677
,person_id,num_date_begin
,Int32?,Int64
1,469917,1475


In [79]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [80]:
size(df_pr_date_min)

(25051, 2)

In [81]:
df_pr_date_min[3015:3021, :]

,person_id,date_min
,Int32?,Int64
1,446883,1434
2,446884,1441
3,446885,1435
4,446886,1437
5,446887,1435
6,446888,1439
7,446889,1436


`date_max`

In [82]:
df_pr_end_value = dropmissing(df_prp_r, :num_date_end)
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id)

,person_id,num_date_end
,Int32?,Int64
1,443845,1690
2,443845,1690
3,443845,1690
,person_id,num_date_end
,Int32?,Int64
1,469917,1525


In [83]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [84]:
size(df_pr_date_max)

(17703, 2)

In [85]:
df_pr_date_max[3015:3021, :]

,person_id,date_max
,Int32?,Int64
1,447786,1470
2,447787,1458
3,447788,1445
4,447789,1438
5,447790,1439
6,447791,1422
7,447792,1411


In [86]:
df_pr_date_min_mng = subset(df_pr_date_min, :person_id => ByRow(ismissing));

In [87]:
size(df_pr_date_min_mng)

(0, 2)

In [88]:
subset!(df_pr_date_min, :person_id => ByRow(!ismissing));

In [89]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [90]:
subset!(df_pr_date_max, :person_id => ByRow(!ismissing));

In [91]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [92]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [93]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [94]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [95]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [96]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [97]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [98]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein

In [99]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [100]:
stmt = DBInterface.prepare(Wds.dbwiag, "UPDATE person SET date_min = ?, date_max = ? WHERE id = ?");

In [101]:
n_loop = 1
for row in eachrow(df_date)
    date_min, date_max, id = row[[:date_min, :date_max, :id]]
    DBInterface.execute(stmt, (date_min, date_max, id))
    n_loop += 1
end
n_loop

25123

In [102]:
DBInterface.close!(stmt)

*2022-01-17* Es gibt etliche Einträge für Personen mit Ämtern nach 2000?! 
```sql
select p.id, id_in_source, item_type_id, givenname, familyname, date_min, date_max, edit_status, is_online
from person as p join item on item.id = p.id and p.date_max > 2000;
```

## Archidiakonat

In [103]:
println.(names(df_prp_r));

Bistum
Amtsart
Amtsbeginn
Amtsende
ID_Amt
Institution
Amt_weltlich
Herrschaftsgebiet
Ort
Amt_Kommentar
Sort
Titularbistum
Profession
ID_Kloster
ID_Fremdschlüssel
Dignität
ID_Domherr
Archidiakonat
Hilfsfeld_import
ID_Amt_norm
id_in_source
person_id
match_role_id
role_name
role_id
diocese_id
institution_id
name
institution_name
num_date_begin
num_date_end
date_sort_key
date_sort_key_replace
id


In [70]:
columns = [
    :id => :person_role_id,
    :person_id => :person_id,
    :Archidiakonat => :value
]

3-element Vector{Pair{Symbol, Symbol}}:
            :id => :person_role_id
     :person_id => :person_id
 :Archidiakonat => :value

In [71]:
df_pr_property = select(df_prp_r, columns);

In [72]:
dropmissing!(df_pr_property, :value);

In [73]:
size(df_pr_property)

(634, 3)

In [74]:
df_pr_property[!, :name] .= "Archidiakonat";

In [75]:
df_pr_property[407:414, :]

,person_role_id,person_id,value,name
,Int64,Int32?,String,String
1,803591,467729,Ostgau,Archidiakonat
2,807228,447916,Dietkirchen,Archidiakonat
3,807229,448279,Dietkirchen,Archidiakonat
4,807230,447939,Karden,Archidiakonat
5,807231,447700,Dietkirchen,Archidiakonat
6,807232,466667,Schmedenstedt,Archidiakonat
7,807233,466663,Bevensen,Archidiakonat
8,807237,466125,Ossendorf,Archidiakonat


In [76]:
table = "role_property_type"
sql = "SELECT * from $(table)"
df_rpt = Wds.sql_df(sql);

In [77]:
df_rpt

,id,name,label,comment,item_type_id
,Int32,String,String,String?,Int32
1,2,Archidiakonat,Archidiakonat,missing,5


In [79]:
rec_archidiakonat = filter(:name => isequal("Archidiakonat"), df_rpt)
archidiakonat_id = rec_archidiakonat[1, :id]

2

In [81]:
insertcols!(
    df_pr_property,
    :property_type_id => archidiakonat_id,
);

,person_role_id,person_id,value,name,property_type_id
,Int64,Int32?,String,String,Int32
1,770298,465483,Mandelsloh,Archidiakonat,2
2,770299,453656,Elze,Archidiakonat,2
3,770300,453713,Hildesheim,Archidiakonat,2
4,770301,453711,Modestorpe,Archidiakonat,2
5,770302,453706,Bevensen,Archidiakonat,2
6,770303,453699,Stöckheim,Archidiakonat,2
7,770304,453696,Sarstedt,Archidiakonat,2
8,770305,453685,Alfeld,Archidiakonat,2
9,770306,453744,Elze,Archidiakonat,2


In [82]:
table_name = "person_role_property";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role_property\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 634, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [83]:
table_name = "person_role_property"
Wds.filltable!(table_name, df_pr_property)

┌ Info: Rows inserted: 634
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


634

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-17* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12